In [ ]:
import pickle
import sagemaker
import boto3
import os
import fxcmpy
import mxnet as mx
import xgboost as xgb
import tarfile
import pickle

In [ ]:
TOKEN = 'ADD_TOKEN_HERE'
con = fxcmpy.fxcmpy(access_token=TOKEN, log_level='error')

In [ ]:
new_data = con.get_candles('EUR/USD', period='H1',number=4000)

In [ ]:
def create_lags_testing(dataset, lags_num):
    #data = dataset.drop(columns = ['bidopen','bidclose', 'bidhigh', 'asklow'] )
    data = dataset.drop(columns = ['askopen','askclose', 'askhigh', 'asklow'] )
    cols = []
    col = ''
    #cols.append('tickqty')
   #print(data.head())
    for lag in range(1, lags_num):
        #col = 'bidhigh_lag_%s' % lag
        #cols.append(col)
        #data[col] = data['bidopen'].shift(lag)
        ##################################################
        col = 'bidclose_15_%s' % lag
        cols.append(col)
        data[col] = data['bidclose'].shift(lag)
        #data[col] = data['bidclose'].shift(lag + 121)
        ##################################################
       # col = 'tick_quant_lag_%s' % lag
        #cols.append(col)
        #data[col] = data['tickqty'].shift(lag)
        
    return data, cols

In [ ]:
alt,columns = create_lags_testing(new_data,90)
alt = alt.dropna()
alt = alt.dropna()

In [ ]:
#new_data2 = new_data.set_index('date', inplace=True, drop=True)
print(new_data.index[:])

In [ ]:
X = new_data['bidclose']
y = alt.iloc[:, 1:2]
X_train = X[:len(X)-30]
X_test = X[len(X)-30:]
y_train = y[:len(y)-30]
y_test = y[len(y)-30:]

In [ ]:
from pandas import read_csv
from pandas import datetime
from matplotlib import pyplot
#import scikits.statsmodels.api as sm
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
 
def parser(x):
	return datetime.strptime('190'+x, '%Y-%m')
 
#series = read_csv('shampoo-sales.csv', header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)
#X = series.values
#size = int(len(X) * 0.66)
#train, test = X[0:size], X[size:len(X)]
history = [x for x in X_train]
#history =  new_data['bidclose']
predictions = list()
for t in range(len(X_test)):
	model = ARIMA(history, order=(5,1,2))
	model_fit = model.fit(disp=0)
	output = model_fit.forecast()
	yhat = output[0]
	predictions.append(yhat)
	obs = X_test[t]
	history.append(obs)
	print('predicted=%f, expected=%f' % (yhat, obs))
error = mean_squared_error(X_test, predictions)
print('Test MSE: %.3f' % error)
# plot
plt.axis()
pyplot.plot(X_test.values)
pyplot.plot(predictions, color='red')
print("Test:",len(X_test), "Prediction:", len(predictions))
#pyplot.set_index(X_test.index[:], inplace=True, drop=True)
pyplot.show()

In [ ]:
%matplotlib inline
import numpy as np
from scipy import stats
import pandas as pd
import matplotlib.pyplot as plt

import statsmodels.api as sm
import statsmodels.api as sm
from statsmodels.graphics.api import qqplot

In [ ]:
history = [x for x in X]
#history =  new_data['bidclose']
predictions = list()
for t in range(len(X_test)):
	model = ARIMA(history, order=(6,0,0))
	model_fit = model.fit(disp=0)
	output = model_fit.forecast()
	yhat = output[0]
	predictions.append(yhat)
	#obs = X_test[t]
	history.append(yhat)
	print('predicted=%f, expected=%f' % (yhat, obs))
#error = mean_squared_error(X_test.values, predictions)
print('Test MSE: %.3f' % error)
# plot
plt.axis()
#pyplot.plot(X_test.values)
pyplot.plot(predictions, color='red')
#print("Test:",len(X_test), "Prediction:", len(predictions))
#pyplot.set_index(X_test.index[:], inplace=True, drop=True)
pyplot.show()

In [ ]:
model.predict(X_train, dynamic = True)[:5]